<a href="https://colab.research.google.com/github/wangzhenyagit/myColab/blob/main/langgraph_code_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 ! pip install -U langchain_community langchain-openai langchain langgraph bs4

## Docs

In [50]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# Langchain docs
url = "https://python.langchain.com/docs/get_started/quickstart/"
loader = RecursiveUrlLoader(
    url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

In [51]:
import os
os.environ['OPENAI_API_KEY'] = 'your_api_key_here'

In [52]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

### OpenAI

# Grader prompt
code_gen_prompt = ChatPromptTemplate.from_messages(
    [("system","""You are a coding assistant with expertise in LangChain. \n
    Here is a full set of LangChain documentation:  \n ------- \n  {context} \n ------- \n Answer the user
    question based on the above provided documentation. Ensure any code you provide can be executed \n
    with all required imports and variables defined. Structure your answer with a description of the code solution. \n
    If code use ChatOpenAI class, do not use the api_key parameter. \n
    Then list the imports. And finally list the functioning code block. Here is the user question:"""),
    ("placeholder", "{messages}")]
)

# Data model
class code(BaseModel):
    """Code output"""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")
    description = "Schema for code solutions to questions about LCEL."

expt_llm = "gpt-4-1106-preview"
llm = ChatOpenAI(temperature=0, model=expt_llm)
code_gen_chain = code_gen_prompt | llm.with_structured_output(code)

In [53]:
question = "How do I ask simple question use langchain?"
solution = code_gen_chain.invoke({"context":concatenated_content,"messages":[("user",question)]})

In [21]:
solution.code

'# Initialize the OpenAI model\nllm = ChatOpenAI(api_key=\'your_openai_api_key\')\n\n# Define a prompt template\nprompt = ChatPromptTemplate.from_messages([\n    ("system", "You are a world class technical documentation writer."),\n    ("user", "{input}")\n])\n\n# Combine the prompt with the model\nchain = prompt | llm\n\n# Add an output parser to convert the chat message to a string\noutput_parser = StrOutputParser()\nchain = chain | output_parser\n\n# Invoke the chain with a simple question\nresponse = chain.invoke({"input": "How can LangSmith help with testing?"})\nprint(response)'

## State

Our state is a dict that will contain keys (errors, question, code generation) relevant to code generation.

In [54]:
from typing import Dict, TypedDict, List

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        error : Binary flag for control flow to indicate whether test error was tripped
        messages : With user question, error messages, reasoning
        generation : Code solution
        iterations : Number of tries
    """

    error : str
    messages : List
    generation : str
    iterations : int

## Graph

Our graph lays out the logical flow shown in the figure above.

In [55]:
from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough

### Parameter

# Max tries
max_iterations = 3

### Nodes

def generate(state: GraphState):
    """
    Generate a code solution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    error = state["error"]

    # We have been routed back to generation with an error
    if error == "yes":
        messages += [("user","Now, try again. Invoke the code tool to structure the output with a prefix, imports, and code block:")]

    # Solution
    code_solution = code_gen_chain.invoke({"context": concatenated_content, "messages" : messages})
    messages += [("assistant",f"{code_solution.prefix} \n Imports: {code_solution.imports} \n Code: {code_solution.code}")]

    # Increment
    iterations = iterations + 1
    return {"generation": code_solution, "messages": messages, "iterations": iterations}

def code_check(state: GraphState):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("---CHECKING CODE---")

    # State
    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]

    # Get solution components
    prefix = code_solution.prefix
    imports = code_solution.imports
    code = code_solution.code

    # Check imports
    try:
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the import test: {e}")]
        messages += error_message
        return {"generation": code_solution, "messages": messages, "iterations": iterations, "error": "yes"}

    # Check execution
    try:
        exec(imports + "\n" + code)
    except Exception as e:
        print("---CODE BLOCK CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the code execution test: {e}")]
        messages += error_message
        return {"generation": code_solution, "messages": messages, "iterations": iterations, "error": "yes"}

    # No errors
    print("---NO CODE TEST FAILURES---")
    return {"generation": code_solution, "messages": messages, "iterations": iterations, "error": "no"}

### Edges

def decide_to_finish(state: GraphState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations == max_iterations:
        print("---DECISION: FINISH---")
        return "end"
    else:
        print("---DECISION: RE-TRY SOLUTION---")
        return "generate"

In [56]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generation solution
workflow.add_node("check_code", code_check)  # check code

# Build graph
workflow.set_entry_point("generate")
workflow.add_edge("generate", "check_code")
workflow.add_conditional_edges(
    "check_code",
    decide_to_finish,
    {
        "end": END,
        "generate": "generate",
    },
)
app = workflow.compile()

In [57]:
question = "How can I use langchain ask question: what is large language model ?"
app.invoke({"messages":[("user",question)],"iterations":0})

---GENERATING CODE SOLUTION---
---CHECKING CODE---
content='A large language model is a type of artificial intelligence system that is trained on vast amounts of text data to understand and generate human-like text. These models are able to process and generate natural language text in a way that is coherent and contextually relevant. Large language models have been used in various applications such as machine translation, text generation, question answering, and more.' response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 28, 'total_tokens': 101}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None} id='run-0f0880ce-cd52-40eb-9406-3e3fa3ede286-0'
---NO CODE TEST FAILURES---
---DECISION: FINISH---


{'error': 'no',
 'messages': [('user',
   'How can I use langchain ask question: what is large language model ?'),
  ('assistant',
   'Using the LangChain library, we can create a simple application to ask a large language model (LLM) a question like \'what is a large language model?\'. The code will include initializing the LLM of choice, creating a prompt template, and invoking the LLM with the question. The following code assumes that the necessary packages have been installed and that the environment variable for the API key has been set as per the documentation. \n Imports: from langchain_openai import ChatOpenAI\nfrom langchain_core.prompts import ChatPromptTemplate \n Code: llm = ChatOpenAI()\n\nprompt = ChatPromptTemplate.from_messages([\n    ("system", "You are an AI trained to answer questions accurately."),\n    ("user", "{input}")\n])\n\nresponse = (prompt | llm).invoke({"input": "what is a large language model?"})\nprint(response)')],
 'generation': code(prefix="Using the 